In [ ]:
import numpy as np
import pandas as pd

from picmol import KBI, ThermoModel, KBIPlotter, PhaseDiagramPlotter

### First Run KBI Analysis as shown in '01_kbi_analysis.ipynb'

In [ ]:
prj_dir = '/path/to/mixture/directory/'
pure_component_dir = '/path/to/pure/component/directory/'

In [ ]:
# initialize kbi class
kbi_obj = KBI(
  prj_path = prj_dir, # location for project to analyze
  pure_component_path = pure_component_dir, # location of pure component directory
  rdf_dir = 'rdf_files', # name for rdf file directory, must be same for all systems
  kbi_method = 'adj', # which kbi correction method to use
  rkbi_min = 0.75, # fraction of rdf to start at for thermo limit extrapolation
  avg_start_time = 100, # when to start averaging properties in .edr file
  solute_mol = '/enter/solute/mol_id/here/', # should be same name as in .top file
  geom_mean_pairs = [], # if desired, enter list of list of molecule pair to use as geom means
)

In [ ]:
# run kbi analysis
kbi_obj.run()

In [ ]:
# create figures
kbi_plotter = KBIPlotter(kbi_obj)
kbi_plotter.make_figures()

### Select a thermodynamic model for LLE analysis

- for any number of components: 'quartic', 'unifac', 'uniquac'
- additional support is provided for only binary mixtures: 'nrtl', 'fh'

In [ ]:
# to evaluate their differences in LLE calculation, use the following (again for just the binary case)
model_dict = {}
models = ['quartic', 'unifac', 'uniquac', 'nrtl', 'fh']

for m, model in enumerate(models):
  tmodel = ThermoModel(
    model_name=model, # name of thermo model to use
    KBIModel=kbi_obj, # kbi object
    Tmin=100, # min temp, for temperature scaling
    Tmax=400, # max temp, for temperature scaling
    dT=5 # temp step, for temperature scaling
    )

  tmodel.run()

  t_plotter = PhaseDiagramPlotter(tmodel)
  t_plotter.make_figures()

  model_dict[model] = tmodel

In [ ]:
# print out their critical point differences
print('Critical point comparisons: ')
for m, model in enumerate(models):
  print(f'{model.upper()}: Tc = { model_dict[model].Tc:0f} K, xc = {model_dict[model].xc:.3f} {kbi_obj.solute_name.lower()}')